In [93]:
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','XP'
dfD.set_index('CR', inplace=True)

dfP = pd.read_csv('../../assets/data/pc-thresholds.csv') # 'Level','Easy XP','Medium XP','Hard XP','Deadly XP','Daily XP'
dfP.set_index('Level', inplace=True)

dfX = pd.read_csv('./xgte-ratios.csv')

In [90]:
# Setup template and html config for plotly figures.
import plotly.graph_objects as go

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=False,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(tickmode='linear', ticks='outside'),
            zeroline=False,
        ),
        yaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(tickmode='linear', ticks='outside'),
            zeroline=False,
        ),
        hoverlabel=dict(align='left'),
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': True, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}

def save_fig_html(fig, format, name, **kwargs):
    file_name = f'./{name}.html'

    match format:
        case 'large':
            fig.update_traces(line_width=2, marker_size=8)
            fig.update_shapes(line_width=2)
            fig.update_layout(font_size=14)
            fig.update_annotations(font_size=14)
        case 'small':
            fig.update_traces(line_width=1, marker_size=6)
            fig.update_shapes(line_width=1)
            fig.update_layout(font_size=10, legend=dict(font=dict(size=10)))
            fig.update_annotations(font_size=10)

    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = f'plotly-div-{format}'
    if kwargs.get('style', None):
        fig_soup.div['style'] = kwargs.get('style')
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [91]:
# Import and process data
import re

# load ratio data
dfX = pd.read_csv('./xgte-ratios.csv')
dfX.set_index('PC level', inplace=True)
dfX = dfX.stack(dropna=False).reset_index()
dfX.rename({'PC level': 'PC level', 'level_1': 'NPC CR', 0: 'ratio'}, axis='columns', inplace=True)
dfX = dfX.astype({'NPC CR': float})

dfX = dfX[~dfX['ratio'].isna()]

# convert to decimal values
def ratio_to_decimal(ratio):
    if ratio is np.nan:
        return ratio
    
    re_ratio = re.compile(r'^(?P<pcs>\d+):(?P<npcs>\d+)$', re.IGNORECASE)
    m = re_ratio.search(ratio)
    return float(m.group('npcs'))/float(m.group('pcs'))

def ratio_to_int(ratio):
    if ratio is np.nan:
        return ratio
    
    re_ratio = re.compile(r'^(?P<pcs>\d+):(?P<npcs>\d+)$', re.IGNORECASE)
    m = re_ratio.search(ratio)
    n_pcs = int(m.group('pcs'))
    n_npcs = int(m.group('npcs'))
    return n_pcs if n_pcs >= n_npcs else -n_npcs

def decimal_to_int(dec):
    return -1.0/dec if np.round(1.0/dec, 0) > 1.5 else dec

def decimal_to_ratio(dec):
    if np.round(1.0/dec, 0) > 1.5:
        n_pc = 1.0/dec
        n_npc = 1
        return f'{n_pc:0.1f}:{n_npc:0.0f}'
    else:
        n_pc = 1
        n_npc = dec
        return f'{n_pc:0.0f}:{n_npc:0.1f}'

dfX['decimal'] = dfX['ratio'].apply(lambda x: ratio_to_decimal(x))
dfX['number'] = dfX['decimal'].apply(lambda x: decimal_to_int(x))

dfX['PC XP']  = dfX['PC level'].apply(lambda x: dfP.at[x, 'Hard XP'])
dfX['NPC XP'] = dfX['NPC CR'].apply(lambda x: 2*dfD.at[x, 'XP'])
#dfX['calc decimal'] = dfX['PC XP']/dfX['NPC XP']
dfX['calc decimal'] = dfX.apply(lambda x: min(12, x['PC XP']/x['NPC XP']), axis=1)
dfX['calc decimal round'] = dfX['calc decimal'].apply(lambda x: np.round(x, 0) if x > 1 else 1/np.round(1/x, 0))
dfX['calc number'] = dfX['calc decimal'].apply(lambda x: decimal_to_int(x))
dfX['calc ratio'] = dfX['calc decimal'].apply(lambda x: decimal_to_ratio(x))

In [61]:
# plots solo monster XP relative to party XP

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0,23],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP ratio',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# add difficulty thresholds
diffs = ['Easy XP', 'Medium XP', 'Hard XP', 'Deadly XP']
colors = ['#1F77B4','#FF7F0E', '#2CA02C', '#D62728']
for diff, color in zip(diffs, colors):
    fig.add_trace(go.Scatter(
        x=dfP.index.to_numpy(), 
        y=2*dfP[diff]/dfP['Daily XP'],
        mode='lines', 
        name=diff,
        showlegend=False,
        line=dict(color=color, dash='dash'),
        hoverinfo='skip',
    ))
    fig.add_trace(go.Scatter(
        x=np.array([20.5]),
        y=np.array(np.mean(2*dfP[diff]/dfP['Daily XP'])),
        mode='text',
        showlegend=False,
        text=[diff.split(' ')[0]],
        textposition='middle right',
        textfont=dict(color=color),
    ))

# load solo monster data
dfS = pd.read_csv('./xgte-solo.csv')
colors = ['rgb(98,83,119)','rgb(104,133,92)','rgb(156,156,94)']
for n_pcs, color in zip([4,5,6], colors):
    col = f'{n_pcs} PCs'
    dfS[f'{col} - PC XP']  = dfS.apply(lambda row: 0.5*n_pcs*dfP.at[row['PC level'], 'Daily XP'], axis=1)
    dfS[f'{col} - NPC XP']  = dfS.apply(lambda row: dfD.at[row[col], 'XP'], axis=1)
    dfS[f'{col} - XP ratio']  = dfS[f'{col} - NPC XP']/dfS[f'{col} - PC XP']
    fig.add_trace(go.Scatter(
        x=dfS['PC level'], 
        y=dfS[f'{col} - XP ratio'],
        mode='markers+lines', 
        line=dict(color=color),
        name=col,
        hovertemplate= 
            f'<b>{col}</b><br>'
            'Level %{x}<br>'+
            'XP ratio %{y:,.2f}' + 
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-solo-monster-xp-ratios-large', style='aspect-ratio: 600/500;')
    save_fig_html(fig, format='small', name='fig-solo-monster-xp-ratios-small', style='aspect-ratio: 600/500;')

In [95]:
# plots solo monster XP relative to party XP (CR + 2)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0,23],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP ratio',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# add difficulty thresholds
diffs = ['Easy XP', 'Medium XP', 'Hard XP', 'Deadly XP']
colors = ['#1F77B4','#FF7F0E', '#2CA02C', '#D62728']
for diff, color in zip(diffs, colors):
    fig.add_trace(go.Scatter(
        x=dfP.index.to_numpy(), 
        y=2*dfP[diff]/dfP['Daily XP'],
        mode='lines', 
        name=diff,
        showlegend=False,
        line=dict(color=color, dash='dash'),
        hoverinfo='skip',
    ))
    fig.add_trace(go.Scatter(
        x=np.array([20.5]),
        y=np.array(np.mean(2*dfP[diff]/dfP['Daily XP'])),
        mode='text',
        showlegend=False,
        text=[diff.split(' ')[0]],
        textposition='middle right',
        textfont=dict(color=color),
    ))

# load solo monster data
dfS = pd.read_csv('./xgte-solo.csv')
colors = ['rgb(98,83,119)','rgb(104,133,92)','rgb(156,156,94)']
for n_pcs, color in zip([4,5,6], colors):
    col = f'{n_pcs} PCs'
    dfS[f'{col} - PC XP']  = dfS.apply(lambda row: 0.5*n_pcs*dfP.at[row['PC level'], 'Daily XP'], axis=1)
    dfS[f'{col} - NPC XP']  = dfS.apply(lambda row: dfD.at[row[col]+2, 'XP'], axis=1)
    dfS[f'{col} - XP ratio']  = dfS[f'{col} - NPC XP']/dfS[f'{col} - PC XP']
    fig.add_trace(go.Scatter(
        x=dfS['PC level'], 
        y=dfS[f'{col} - XP ratio'],
        mode='markers+lines', 
        line=dict(color=color),
        name=col,
        hovertemplate= 
            f'<b>{col}</b><br>'
            'Level %{x}<br>'+
            'XP ratio %{y:,.2f}' + 
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-solo-monster-xp-ratios-harder-large', style='aspect-ratio: 600/500;')
    save_fig_html(fig, format='small', name='fig-solo-monster-xp-ratios-harder-small', style='aspect-ratio: 600/500;')

In [94]:
# plots solo monster XP relative to party XP (CR + 2)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0,23],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP ratio',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# add difficulty thresholds
diffs = ['Easy XP', 'Medium XP', 'Hard XP', 'Deadly XP']
colors = ['#1F77B4','#FF7F0E', '#2CA02C', '#D62728']
for diff, color in zip(diffs, colors):
    fig.add_trace(go.Scatter(
        x=dfP.index.to_numpy(), 
        y=2*dfP[diff]/dfP['Daily XP'],
        mode='lines', 
        name=diff,
        showlegend=False,
        line=dict(color=color, dash='dash'),
        hoverinfo='skip',
    ))
    fig.add_trace(go.Scatter(
        x=np.array([20.5]),
        y=np.array(np.mean(2*dfP[diff]/dfP['Daily XP'])),
        mode='text',
        showlegend=False,
        text=[diff.split(' ')[0]],
        textposition='middle right',
        textfont=dict(color=color),
    ))

# load solo monster data
dfS = pd.read_csv('./xgte-solo.csv')
dfS = dfS[dfS['PC level'].between(4,20)]
colors = ['rgb(98,83,119)','rgb(104,133,92)','rgb(156,156,94)']
for n_pcs, color in zip([4,5,6], colors):
    col = f'{n_pcs} PCs'
    dfS[f'{col} - PC XP'] = dfS.apply(lambda row: 0.5*n_pcs*dfP.at[row['PC level'], 'Daily XP'], axis=1)
    dfS[f'{col} - NPC XP'] = dfS.apply(lambda row: dfD.at[max(row[col]-3, 0.5), 'XP'], axis=1)
    dfS[f'{col} - XP ratio'] = dfS[f'{col} - NPC XP']/dfS[f'{col} - PC XP']
    fig.add_trace(go.Scatter(
        x=dfS['PC level'], 
        y=dfS[f'{col} - XP ratio'],
        mode='markers+lines', 
        line=dict(color=color),
        name=col,
        hovertemplate= 
            f'<b>{col}</b><br>'
            'Level %{x}<br>'+
            'XP ratio %{y:,.2f}' + 
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-solo-monster-xp-ratios-easier-large', style='aspect-ratio: 600/500;')
    save_fig_html(fig, format='small', name='fig-solo-monster-xp-ratios-easier-small', style='aspect-ratio: 600/500;')

In [54]:
print(list(dfX['ratio'].unique()))
print(list(dfX['decimal'].unique()))

tickvals=[2.0, 1.0, 0.3333333333333333, 0.2, 3.0, 0.16666666666666666, 5.0, 0.5, 0.25, 8.0, 4.0, 12.0, 9.0, 6.0, 7.0, 10.0]
ticktext=['1:2', '1:1', '3:1', '5:1', '1:3', '6:1', '1:5', '2:1', '4:1', '1:8', '1:4', '1:12', '1:9', '1:6', '1:7', '1:10']



['1:2', '1:1', '3:1', '5:1', '1:3', '6:1', '1:5', '2:1', '4:1', '1:8', '1:4', '1:12', '1:9', '1:6', '1:7', '1:10']
[2.0, 1.0, 0.3333333333333333, 0.2, 3.0, 0.16666666666666666, 5.0, 0.5, 0.25, 8.0, 4.0, 12.0, 9.0, 6.0, 7.0, 10.0]


In [92]:
# plots XGtE ratios vs calculated ratios
tickvals=[2.0, 1.0, 0.3333333333333333, 0.2, 3.0, 0.16666666666666666, 5.0, 0.5, 0.25, 8.0, 4.0, 12.0, 9.0, 6.0, 7.0, 10.0]
ticktext=['1:2', '1:1', '3:1', '5:1', '1:3', '', '1:5', '2:1', '', '', '', '1:12', '1:9', '', '1:7', '']

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='XGtE ratio',
            type='log',
            tickvals=tickvals,
            ticktext=ticktext,
        ),
        yaxis=dict(
            title_text='calculated ratio',
            type='log',
            tickvals=tickvals,
            ticktext=ticktext,
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

def make_hovertext(row):
    ratio = row['ratio']
    cratio = row['calc ratio']
    text = f'<b>PCs : Monsters</b><br>XGtE {ratio}<br>Calculated {cratio}'
    return text
    
hovertext = dfX.apply(lambda row: make_hovertext(row), axis=1)

# plot conversion ratios
fig.add_trace(go.Scatter(
    x=dfX['decimal'], 
    y=dfX['calc decimal'],
    mode='markers', 
    name='Calculated monsters per PC',
    showlegend=False,
    hovertext=hovertext,
    hovertemplate='%{hovertext}<extra></extra>',
))

coefs = np.polyfit(dfX['decimal'], dfX['calc decimal'], 1)
poly = np.poly1d(coefs)
fig.add_trace(go.Scatter(
    x=[0,13], 
    y=[0,13],
    mode='lines', 
    name='fit',
    showlegend=False,
    line=dict(color='black', dash='dash'),
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-multi-monster-ratios-large')
    save_fig_html(fig, format='small', name='fig-multi-monster-ratios-small')

In [63]:
# plots threshold XP relative to the Hard XP threshold

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP ratio',
            range=[0,1.8],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.02,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

diffs = ['Medium XP', 'Hard XP', 'Deadly XP']
colors = ['#FF7F0E', '#2CA02C', '#D62728']
for diff, color in zip(diffs, colors):
    fig.add_trace(go.Scatter(
        x=dfP.index.to_numpy(), 
        y=dfP[diff]/dfP['Hard XP'],
        mode='lines+markers', 
        name=diff,
        line=dict(color=color),
        hovertemplate= 
            f'<b>{diff}</b><br>'
            'Level %{x}<br>'+
            'XP ratio %{y:,.2f}' + 
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-xp-threshold-ratios-large')
    save_fig_html(fig, format='small', name='fig-xp-threshold-ratios-small')

In [66]:
def encounter_multiplier_DMG(pc_count, npc_count):
    """Returns the encounter multiplier given by the DMG
    pc_count -- number of PCs in the encounter
    npc_count -- number of NPCs in the encounter
    """
    n_array = np.asarray([1,2,3,7,11,15])
    m_array = np.asarray([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0])
    i = 1 + n_array[n_array <= max(npc_count,1)].argmax()
    if pc_count >= 6:
        i -= 1
    elif pc_count <= 2:
        i += 1
    return m_array[i]


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='NPCs',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='encounter multiplier',
            range=[0,4.5],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot encounter multiplier
x = np.linspace(1, 20, 20)
y = np.array([encounter_multiplier_DMG(4, n) for n in x])
fig.add_trace(go.Scatter(
    x=x, 
    y=y,
    mode='markers+lines', 
    name='1 monster per group (Basic Rules)',
    hovertemplate= 
        f'<b>Basic Rules</b><br>'
        'NPCs %{x}<br>'+
        'Multiplier %{y:,.2f}' + 
        '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=x, 
    y=0.5*y,
    mode='markers+lines', 
    name='1 monster per PC (XGtE)',
    hovertemplate= 
        f'<b>XGtE</b><br>'
        'NPCs %{x}<br>'+
        'Multiplier %{y:,.2f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-encounter-multiplier-large')
    save_fig_html(fig, format='small', name='fig-encounter-multiplier-small')